# ELEC474 Project

In [189]:
import cv2 as cv
import numpy as np
import os
from cv2 import Stitcher
from matplotlib import pyplot as plt
from imutils.perspective import four_point_transform

In [190]:
global my_SIFT_instance, my_BF_instance
my_SIFT_instance = cv.SIFT_create()

FLANN_INDEX_KDTREE = 0 #heard from C++ api that this should be 1
# FLANN_INDEX_KDTREE = 1

dirName1 = 'office2'
dirName2 = 'StJames'
dirName3 = 'WLH'

imgDescipt_1 = np.array((
"Left Key Point",
"Right Key Points"
))

imgDescipt_2 = np.array((
"Select point Output",
"Epipolar Line Output"
))

Process_BarLength = 30

BEST_MATCH_METRIC = 300 #StJames is 85, office2 is 200, WLH is 100
LOWE_RATIO = 0.7
SEED_IDX = 14

# Step 1

### Class Def

In [191]:
class MatcheClass:
    """[summary]
    \nThis Class is used to store and transfer keypoints, descriptors, images between functions
    """
    def __init__(self, kp, des):
        self.kp = kp
        self.des = des
    
    def LoadMatch(self, match):
        self.matchs = match
    
    def LoadImg(self, img):
        self.img = img
    
    def __eq__(self, other) -> bool:
        return self.__dict__ == other.__dict__
        

### Tool Functions

In [192]:
def PltImg(img,imgDescipt):
    """
    \nTake in a img list and a img descriptions to do plt.imshow in a window
    \nWith dip == 300 and size zoom (15,15)
    Args:
        img ([list]): Remember put a single img in list e.g. PltImg([img], ["Description"])
        imgDescipt ([List]): List of String to descripe each img
    """
    plt.figure(dpi=300)
    plt.figure(figsize=(15,15))
    idx = len(img)
    for i in range(idx):
        plt.subplot(1,idx,i+1)

        if(len(img[i].shape) == 2): #differ from gray and color img 
            plt.imshow(img[i],cmap="gray")
        else:
            plt.imshow(cv.cvtColor(img[i], cv.COLOR_BGR2RGB))

        plt.title(imgDescipt[i])
    plt.tight_layout()

def ReadImgs(dirName, imgName):
    """
    \nFetching images based on given directory and return it 
    Args:
        dirName (string): name of dir 
        imgName (string): name of img]
    """
    return cv.imread(os.getcwd()+'//'+dirName+'//'+imgName)

def FetchingImgs(dirName):
    """Fetching Imgs based on given file name
    \n Remember the file should be in the same dir with this script 
    Args:
        dirName (string): Name of the file

    Returns:
        [List]: a list of imgs
    """
    #list all the img file under dir 
    ls = []
    dir = os.getcwd()+'//'+dirName
    files = os.listdir(dir)
    for filename in files:
        # print(dir + os.sep + filename)

        if os.path.splitext(filename)[1] == '.jpg':
            ls.append(filename)
    if ls.count != 0:
        print("Detected", len(ls),"Images")
    return ls

def ProgressionBarUpdate(current,overall):
    """\n Tools used to show update by a progression 
    Args:
        current (int): current progress
        overall (int): total progress
    """
    pctge = (current+1) / overall
    if pctge > 1:
        pctge = 1
    temp = int(round(Process_BarLength * pctge))
    print('\r%s%s%s%s'% ((temp)*'█',(Process_BarLength-temp)*'░',str(round(pctge*100)),'%'), end = ' ')


### Matching function class

In [193]:
def FindDescriptorAndKeyPoints(img,descriptor,flag):
    """find key points and descriptors based on given imgs and descriptor instance
    Args:
        flag ([int]): when img is BGR image, set flag to 1, if gray set to 0
    Returns:
        [MatcheCLass]
    """
    
    imgGray = img
    if flag == 1:
        imgGray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    kp, des = descriptor.detectAndCompute(imgGray,None)
    return MatcheClass(kp, des)

def FlannBasedMatchLoweRatio(descriptor1,descriptor2,   
                            flann_Instance,
                            kNum = 2, ratio = 0.7):
    """
    Args:
        kNum (int, optional): [description]. Defaults to 2.
        ratio (float, optional): [description]. Defaults to 0.7.

    Returns:
        [Match]
    """
    
    matches = flann_Instance.knnMatch(descriptor1,descriptor2,k = kNum)

    loweMatch = []
    for m,n in matches:
        if m.distance < ratio * n.distance:
            loweMatch.append(m)
    return loweMatch

def Matching(img1, img2, descriptor = my_SIFT_instance, loweRatio = LOWE_RATIO):
    """[summary]
    With given 2 imgs return there kps des, and matchs  
    Args:

        descriptor ([type], optional): Defaults to my_SIFT_instance.
        loweRatio ([type], optional):Defaults to LOWE_RATIO.

    Returns:
        [MatcheClass]: with only kp and des inside, usally for seed img
        [MatcheClass]: with all inside and their matchs
    """
    
    #FLANN para and create
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)       #or pass empty dict #It specifies the number of times the trees in the index should be recursively traversed.
    flann_Instance = cv.FlannBasedMatcher(index_params,search_params)
    
    #Find img is colored or gray scale
    flag = 0
    if(len(img1.shape) == 3):
        flag = 1
    
    kpDes_1 = FindDescriptorAndKeyPoints(img1, descriptor, flag)
    kpDes_2 = FindDescriptorAndKeyPoints(img2, descriptor, flag) 
    matches = FlannBasedMatchLoweRatio(kpDes_1.des, kpDes_2.des,
                            flann_Instance,
                            kNum = 2, ratio = loweRatio)  #NOTE: The ratio difference of lowe will result in different epiploar lines 
    
    kpDes_2.LoadMatch(matches)
    kpDes_2.LoadImg(img2)
    return kpDes_1, kpDes_2

In [194]:

def MatchingFeaturesOfImages(dirName, descriptorMethod = 'sift'):
    """[summary]
    \nWith given directory name, \nFetching imgs and find the best imgs with given seed img and good match imgs
    Which control by SEED_IDX and BEST_MATCH_METRIC\n
    Args:
        dirName (string): name of the file with all imgs inside, BESURE it is in the same dir with this script
        descriptorMethod (str, optional):Can use sift, surf and brisk. Defaults to 'sift'.

    Returns:
        [MatcheClass]: seed MatcheClassimg
        [MatcheClass List]: good match MatcheClass lists 
    """
    
    
    idx = 0
    numGoodMatch = 0
    imgList = []
    matchList = []

    # detect and extract features from the image
    if descriptorMethod == 'sift':
        descriptor = cv.xfeatures2d.SIFT_create()
    elif descriptorMethod == 'surf':
        descriptor = cv.xfeatures2d.SURF_create()
    elif descriptorMethod == 'brisk':
        descriptor = cv.BRISK_create()
    
    print("Featching Images under Dir and creating descriptors for them")
    imgNameList = FetchingImgs(dirName)
    for imgName in imgNameList:
        idx += 1
        ProgressionBarUpdate(idx, len(imgNameList)+1)
        
        temp = ReadImgs(dirName, imgName)
        imgList.append(temp)
        

    seedIdx = SEED_IDX
    seedImg = imgList[seedIdx]
    print("\nYour Seed Idex from Imgaes is", SEED_IDX, "And Your Seed Image Looks like:")
    PltImg([seedImg],["Seed Image"])
    
    print("\nFinding best Match for each pair of imgs")
    imgListIdx = list(range(len(imgList)))
    imgListIdx.remove(seedIdx)
    for imgIdx in imgListIdx:
        ProgressionBarUpdate(imgIdx+1,len(imgList)-1)
        seedPara,compare = Matching(seedImg, imgList[imgIdx],descriptor)
        if len(compare.matchs) > BEST_MATCH_METRIC:
            matchList.append(compare)
            numGoodMatch += 1
    seedPara.LoadImg(seedImg)
            
    
    print("\nThere are", numGoodMatch, "good Image Pairs has been find")
        
    return seedPara,matchList

        

# Step 2,3

In [195]:
def RLListClasification(mList:MatcheClass, seedPara:MatcheClass):
    """[summary]
    Divde imgs into right and left imgs based on seed imgs
    Args:
        mList (MatcheClass): list of good match MatcheClass
        seedPara (MatcheClass): seed img's MatcheClass

    Returns:
        [list]: list of MatcheClass belongs to LEFT  side of seed 
        [list]: list of MatcheClass belongs to RIGHT side of seed 
    """
    
    leftList = []
    rightList = []
    for currentPair in mList:
        
        height_src, width_src = seedPara.img.shape[:2]
        H = FindHomographic(seedPara.kp, currentPair.kp,currentPair.matchs)
        pts = np.float32(
        [[0, 0], [0, height_src], [width_src, height_src], [width_src, 0]]
        ).reshape(-1, 1, 2)
        direction = cv.perspectiveTransform(pts, H)
        
        if direction[0][0][0] < 0: 
            # print("is Left Image")
            # PltImg([seedImg, cmprImg], ["L compare", "L source"])
            leftList.append(currentPair)
        else:
            # print("is Right Imgae")
            rightList.append(currentPair)
            # PltImg([seedImg, cmprImg], ["R compare", "R source"]
            
    #Sort from large matchs to low matches
    leftList.sort(key = lambda para:len(para.matchs), reverse=False) 
    rightList.sort(key = lambda para:len(para.matchs), reverse=True) 
    return leftList, rightList

def FindHomographic(kp1, kp2, match):
    """[summary]
    \nReturn H matrix based on given kypoints
    """
    ref_pts = np.float32([kp1[m.queryIdx].pt for m in match]).reshape(-1,1,2)
    img_pts = np.float32([kp2[m.trainIdx].pt for m in match]).reshape(-1,1,2)

    H, _ = cv.findHomography(ref_pts, img_pts, cv.RANSAC, 5)
    
    return H


In [196]:
def ChopBlackSide(img):
    """[summary]
    \nBased on given img, chop out the blackside without effective pixels 
    """
    
    image = img 
    img = cv.medianBlur(image, 5)
    gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    _, th = cv.threshold(gray_img, 1, 255, cv.THRESH_BINARY)
    contour, _ = cv.findContours(th, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnt = contour[0]
    x,y,w,h = cv.boundingRect(cnt)
    cropImg = img[y:y+h, x:x+w]
    # PltImg([cropImg],"Chop")
    return cropImg

def StichImgesPair(imgMain, imgWarp):
    """[summary]
    \nWith given 2 imgs return their stich img
    Args:
        imgMain ([Img]): The img that will be stick back, with no warp at all
        imgWarp ([Img]): The img that will be warped and overlay by imgMain
    """
    warpBuffer = [imgMain.shape[1] + imgWarp.shape[1], imgMain.shape[0]+imgWarp.shape[0]]
    img1Para, img2Para = Matching(imgMain, imgWarp)
    H = FindHomographic(img1Para.kp, img2Para.kp,img2Para.matchs)
    H = np.linalg.inv(H)
    
    warpImg = cv.warpPerspective(imgWarp, H, warpBuffer)
    warpImg[0:imgMain.shape[0], 0:imgMain.shape[1]] = imgMain
    warpImg = ChopBlackSide(warpImg)
    PltImg([warpImg, imgMain, imgWarp], ["Warped", "Source", "Append"]) #Uncomment this to see by steps
    
    return warpImg

In [197]:
def FinalRefine(img, offset):
    """[summary]
    \nThe raw result has many deformation, based on raw return a better result
    Args:
        offset (int): Usually be the original img height
    """
    height,width = img.shape[:2]
    top_left = [0,0]
    o_top_right = [width, 0]
    bottom_right = [width,height]
    bottom_left = [0,offset]
    point = np.array((top_left, bottom_left, o_top_right, bottom_right),dtype = "float32")
    img = four_point_transform(img, point)
    return img

In [198]:
def StichImgMainFunc(dirName):
    """[summary]
    \nThe Main fucntion for this script, with given folder name Plt the stich imgs
    Args:
        dirName ([string]): img folder name
    """
    print("Start Fetching images under Dir", dirName)
    seedPara, mList = MatchingFeaturesOfImages(dirName)
    print("Classify Images into Left And Right based on Seed Image")
    leftList, rightList = RLListClasification(mList, seedPara)

    print("Start Stiching Imgs together")
    warpComb = StichImgesPair(leftList[0].img, seedPara.img)
    print("Start Stiching With Left pairs ....")
    idx = 0
    for currentLeft in leftList[1:]:
        idx += 1
        ProgressionBarUpdate(idx,len(leftList))
        warpComb = StichImgesPair(currentLeft.img, warpComb) 
    print("\nCalabrating ......")
    warpComb = FinalRefine(warpComb, seedPara.img.shape[0])
    # PltImg([warpComb], ["Check"])
    print("\nStart Stiching With Right pairs ....")
    idx = 0
    for currentRight in rightList[:]:
        idx += 1
        ProgressionBarUpdate(idx,len(rightList))
        warpComb = StichImgesPair(currentRight.img,warpComb)
        # print("\nCalabrating ......")
        # warpComb = FinalRefine(warpComb, seedPara.img.shape[0])
    PltImg([warpComb], ["Raw Result"])
    print("\nStart Final Calabration")
    warpComb = FinalRefine(warpComb, seedPara.img.shape[0])
    
    print("!!!!!FINISH!!!!!")
    PltImg([warpComb], ["FINAL"])

In [199]:
StichImgMainFunc(dirName2)

Start Fetching images under Dir StJames
Featching Images under Dir and creating descriptors for them
Detected 16 Images
██████████████████████████████100% 
Your Seed Idex from Imgaes is 14 And Your Seed Image Looks like:

Finding best Match for each pair of imgs
██████░░░░░░░░░░░░░░░░░░░░░░░░20% 